# 09-01 Redis 基础

Redis 是内存数据库，常用于缓存、会话、消息队列。OpenClaw 使用 Redis 存储会话状态。

## 1. 连接与基础操作

In [ ]:
import { createClient } from 'redis';

const client = createClient({
  url: 'redis://localhost:6379'
});

client.on('error', (err) => console.error('Redis Error:', err));

await client.connect();

// 字符串操作
await client.set('name', 'Alice');
const name = await client.get('name');
console.log(name);  // 'Alice'

// 设置过期时间（秒）
await client.setEx('temp', 60, 'expires in 60s');

// 删除
await client.del('name');

## 2. 哈希（对象存储）

In [ ]:
// 存储用户对象
await client.hSet('user:1', {
  name: 'Alice',
  email: 'alice@example.com',
  age: '25'
});

// 获取整个对象
const user = await client.hGetAll('user:1');
console.log(user);
// { name: 'Alice', email: 'alice@example.com', age: '25' }

// 获取单个字段
const email = await client.hGet('user:1', 'email');

// 增加数字字段
await client.hIncrBy('user:1', 'loginCount', 1);

## 3. 列表（队列）

In [ ]:
// 消息队列
await client.lPush('queue:messages', 'msg1');
await client.lPush('queue:messages', 'msg2');

// 从右侧弹出（FIFO）
const msg = await client.rPop('queue:messages');
console.log(msg);  // 'msg1'

// 阻塞弹出（等待新消息）
const result = await client.brPop('queue:messages', 5);  // 等待5秒

## 4. 集合与有序集合

In [ ]:
// Set（唯一值）
await client.sAdd('tags:post:1', ['nodejs', 'redis', 'tutorial']);
await client.sAdd('tags:post:2', ['nodejs', 'javascript']);

// 交集
const common = await client.sInter(['tags:post:1', 'tags:post:2']);
console.log(common);  // ['nodejs']

// Sorted Set（带分数）
await client.zAdd('leaderboard', [
  { score: 100, value: 'player1' },
  { score: 200, value: 'player2' },
  { score: 150, value: 'player3' }
]);

// 获取前10名
const top = await client.zRangeWithScores('leaderboard', 0, 9, { REV: true });
console.log(top);
// [{ value: 'player2', score: 200 }, { value: 'player3', score: 150 }, ...]

## 5. 缓存模式

In [ ]:
// Cache-Aside 模式
async function getUser(userId) {
  const cacheKey = `user:${userId}`;
  
  // 先查缓存
  const cached = await client.get(cacheKey);
  if (cached) {
    return JSON.parse(cached);
  }
  
  // 缓存未命中，查数据库
  const user = await db.users.findById(userId);
  
  // 写入缓存（5分钟过期）
  await client.setEx(cacheKey, 300, JSON.stringify(user));
  
  return user;
}

// 删除时使缓存失效
async function deleteUser(userId) {
  await db.users.delete(userId);
  await client.del(`user:${userId}`);
}

## OpenClaw 会话存储

In [ ]:
// OpenClaw 风格的会话存储
class SessionStore {
  constructor(redis) {
    this.redis = redis;
  }
  
  async getSession(sessionId) {
    const data = await this.redis.hGetAll(`session:${sessionId}`);
    if (!data || Object.keys(data).length === 0) {
      return null;
    }
    return {
      id: sessionId,
      userId: data.userId,
      channel: data.channel,
      createdAt: new Date(data.createdAt),
      lastActivity: new Date(data.lastActivity)
    };
  }
  
  async saveSession(sessionId, session) {
    await this.redis.hSet(`session:${sessionId}`, {
      userId: session.userId,
      channel: session.channel,
      createdAt: session.createdAt.toISOString(),
      lastActivity: new Date().toISOString()
    });
    // 7天过期
    await this.redis.expire(`session:${sessionId}`, 7 * 24 * 60 * 60);
  }
}

## 练习

1. 实现一个 Rate Limiter（限制 API 调用频率）
2. 实现分布式锁
3. 查看 OpenClaw 中的 Redis 使用